In [1]:
import numpy as np
import pandas as pd
import keras
import eng_to_ipa as ipa
from nltk.tokenize import PunktSentenceTokenizer
from nltk import word_tokenize
from nltk import pos_tag
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [2]:
df = pd.read_csv('Multiple_Forms.csv', index_col = 0)
df.head()

,Author,Poem,Form
0,William Shakespeare,"From fairest creatures we desire increase,\r\n...",Sonnet
1,William Shakespeare,"When forty winters shall besiege thy brow,\r\n...",Sonnet
2,William Shakespeare,Look in thy glass and tell the face thou viewe...,Sonnet
3,William Shakespeare,"Unthrifty loveliness, why dost thou spend\r\n ...",Sonnet
4,William Shakespeare,"Those hours, that with gentle work did frame\r...",Sonnet


In [3]:
poems = df['Poem']
poems.head(20)

0     From fairest creatures we desire increase,\r\n...
1     When forty winters shall besiege thy brow,\r\n...
2     Look in thy glass and tell the face thou viewe...
3     Unthrifty loveliness, why dost thou spend\r\n ...
4     Those hours, that with gentle work did frame\r...
5     Then let not winter's ragged hand deface,\r\n ...
6     Lo! in the orient when the gracious light\r\n ...
7     Music to hear, why hear'st thou music sadly?\r...
8     Is it for fear to wet a widow's eye,\r\n  That...
9     For shame! deny that thou bear'st love to any,...
10    As fast as thou shalt wane, so fast thou grow'...
11    When I do count the clock that tells the time,...
12    O! that you were your self; but, love you are\...
13    Not from the stars do I my judgement pluck;\r\...
14    When I consider every thing that grows\r\n  Ho...
15    But wherefore do not you a mightier way\r\n  M...
16    Who will believe my verse in time to come,\r\n...
17    Shall I compare thee to a summer's day?\r\

In [4]:
train_poems, sample_poems = train_test_split(poems, test_size=0.7)

In [5]:
t_text = ''
for poem in train_poems:
    t_text += str(poem) + ' '
train_text = t_text.replace('\n','')
len(train_text)

231906

In [6]:
s_text = ''
for poem in sample_poems:
    s_text += str(poem) + ' '
sample_text = s_text.replace('\n', '')
len(sample_text)

548540

In [7]:
# train the Punkt tokenizer
custom_sent_tokenizer = PunktSentenceTokenizer(train_text)

In [8]:
# tokenize using the sample text
tokenized = custom_sent_tokenizer.tokenize(sample_text)
print(len(tokenized))

4250


In [9]:
print(tokenized)

['nan\nnan\nnan\nnan\nnan\nMarch, march, Ettrick and Teviotdale, \nWhy the deil dinna ye march forward in order!', 'March, march, Eskdale and Liddesdale, \nAll the Blue Bonnets are bound for the Border.', 'Many a banner spread,\nFlutters above your head, \nMany a crest that is famous in story.', 'Mount and make ready then, \nSons of the mountain glen, \nFight for the Queen and our old Scottish glory.', 'Come from the hills where your hirsels are grazing, \nCome from the glen of the buck and the roe; \nCome to the crag where the beacon is blazing, \nCome with the buckler, the lance, and the bow.', 'Trumpets are sounding, \nWar-steeds are bounding, \nStand to your arms, then, and march in good order; \nEngland shall many a day \nTell of the bloody fray, \nWhen the Blue Bonnets came over the Border.', "'DARK gathering clouds involve the threatening skies,\nThe sea heaves conscious of the impending gloom,\nDeep, hollow murmurs from the cliffs arise;\nThey come--the Spirits of the Tempest c

In [10]:
try:
    tagged = []
    for i in tokenized:
        i = i.strip('nan\n')
        words = word_tokenize(i)
        tagged.extend(pos_tag(words))
    print(tagged[0])
except Exception as e:
    print(e)

('March', 'NNP')


In [18]:
word_list = list(tagged)
len(word_list)
print(word_list[0:20])

[('March', 'NNP'), (',', ','), ('march', 'NN'), (',', ','), ('Ettrick', 'NNP')]


In [19]:
maxlen = 6                                                            
step = 1                                                               

sentences = []                                                         

next_words = []                                                        

for i in range(0, len(word_list) - maxlen, step):
    sentences.append(word_list[i: i + maxlen])
    next_words.append(word_list[i + maxlen])

print('Number of sequences:', len(sentences))

words = sorted(list(set(word_list)))
print(words[0:5])
print('Unique words:', len(words))
word_indices = dict((word, words.index(word)) for word in words)       

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)      
y = np.zeros((len(sentences), len(words)), dtype=np.bool)              
for i, sentence in enumerate(sentences):                               
    for t, word in enumerate(sentence):                                
        x[i, t, word_indices[word]] = 1                                
    y[i, word_indices[next_words[i]]] = 1 

Number of sequences: 114786
[('!', '.'), ('&', 'CC'), ("'", "''"), ("'", 'POS'), ("''", "''")]
Unique words: 17910
Vectorization...


In [13]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(words))))
model.add(layers.Dense(len(words), activation='softmax'))

In [14]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [15]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [33]:
import random
import sys

generated_text = ''
count = 0

for epoch in range(1, 21):
    generated_text = ''
    generated_list = []
    print('\nepoch', epoch)
    model.fit(x, y, batch_size=128, epochs=1)                       
    start_index = random.randint(0, len(word_list) - maxlen - 1)         
    generated_list = word_list[start_index: start_index + maxlen]
#     generated_list = ['The', 'cat', 'in', 'hat']
    for word in generated_list:
        print(word)
        generated_text += word[0] + ' '
    generated_text =  generated_text.strip()
    print('--- Generating with seed: "' + generated_text + '"')
    for num in ipa.syllable_count(generated_text):
        count += num
    for temperature in [0.2, 0.5, 1.0, 1.2]:                        
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        for i in range(400):                                        
            sampled = np.zeros((1, maxlen, len(words)))             
            for t, word in enumerate(generated_list):               
                sampled[0, t, word_indices[word]] = 1.              

            preds = model.predict(sampled, verbose=0)[0]            
            next_index = sample(preds, temperature)                 
            next_word = words[next_index]                           

            generated_list.append(next_word)
            generated_list = generated_list[1:]

            sys.stdout.write(' ' + next_word[0])
            if ipa.syllable_count(next_word[0]) == 0:
                if len(next_word[0]) >= 9:
                    count += 3
                elif len(next_word[0]) >= 6:
                    count += 2
                elif len(next_word[0]) == 1:
                    count += 0
                else:
                    count += 1
            else:
                count += ipa.syllable_count(next_word[0])

            if count >= 10:
                sys.stdout.write('\n')
                count = 0


epoch 1
Epoch 1/1
114786/114786 [==============================] - 754s 7ms/step - loss: 6.9124
(':', ':')
('This', 'DT')
('is', 'VBZ')
('the', 'DT')
('end', 'NN')
('of', 'IN')
--- Generating with seed: ": This is the end of"
------ temperature: 0.2
: This is the end of every man 's
 desire . The burden of his step , And the
 sea and the , of the sea , And , and the , and
 , as a , and , of the , and , , and , , and , , and , , to
 the , and the , of the , and , , and the , and the
 , of the , and they , and the , of love , I have
 , to love , and to make , Nor , With a , and , as
 a , and , and the , of the , and her , With a
 and eye , and her love , and , With a man , and
 the , and the , sea , and the sea , And all her
 head , and the , of a , as a of love , And
 I am my love , I have I , as , to the
 , and I , and , , and , as a , and , and the , of
 , , and , and , , and , , and the , and the , With a and
 , and the , With and a , or , or , , or , , and , , and
 , , and the , an

 sea , and the sea , And the sea and the sea
 , And the sea and the sea , And the sea and
 the sea , And the sea , and the sea , And the
 sea and the sea , And the sea and the sea
 , And the sea and the sea , And------ temperature: 0.5
journey from my Delia 's side the sea and
 plain , When the , which , and long , and night , And I
 have heard , And his eyes are down and We were
 the sea , And in his face was given : The
 old man can take a face of Or , And the
 men , who , as we were Of the sea , and Lost
 the night Of the night , and , or the sea , And
 have It was like a king , And the dead like
 a child Of the sound , the voice Of like the
 sea , And the sea , our and by , and only
 His eyes , And the sea and lie , And never
 saw the men who died With the night Of night
 , as the night , and the sea , And the sea and
 the sea , And the wind and the sea , It was
 like the sea , And the sea and came , And all
 our night , And at our two eyes , And his eyes
 was down , And the and se

 head , And the You of the years , I know not
 's then , how all my love was , though the 's
 days are happy now , And all things of the
 heart of the sun that is the now in the
 way , for the hearts of gold peace , Thy its heart
 , in , nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan------ temperature: 1.0
can but admire And offer you nan Henry thy truth to
 All that place Wit

midnight , he and I . -- when , I bring not been , To make me . You
 grace me I have have dog-star me , And Alfred
 , she , and ’ plain ; But when black ( has de ? '' And
 if the call them -- break love , and home young
 , And often me had but Whilst had With each
 house and from her The crown ; And you at breath
 of you have I : , but which shall Here to be
 sea-lane aspect ? What thou world so soon Who
 wit rival us which to make ? O ! one must
 see that what Ever which subject to his
 many lyre , And when his she correll memory
 . there , sweet By thy thine , Dragon more me thy
 true ; beauty shall be so long the time Which
 Like no some trouble did from . The world that
 've a but through still house And He never
 kept dog of his he , And beauty 's thou
 fight to the last look . With now a world child
 thee , neither tears own some Some mantles , Of
 faire black cloud-sheaves Rich come to be With
 Peace , no blue , there think the loss of thee , And
 born of stone before an beaux w

 not to the King , And where thou is not his
 father . The 's faire time not 'd love in
 for me . I only am ; but now the not
 hold one thy idiot we The gypsies
 that you have made thee with all all much high
 , who best for victories , for me a your
 name the I go . But the below hole ( for
 their ) and makes thy here . be , O nothing , how
 so but you has you gone , But I was not
 , but his life were not . I came to prove the
 Regulations But : The not said great , Of
 foot my wits and divine , m I do not
 here ? There is not days when her cheeks from her
 height palm , You are not with thy hand . Thus for
 the me to the that thou war-ship water
 , do And from the his face of eyes , His desire
 , The mellow takes me , did not part thy Thou
 nan me ; be shall no find------ temperature: 1.2
store ; Death comes , and you fate The man
 so true That summer ? , have threshold my bonds
 the yourself , who by your room , then not from
 aid of the leaves you wear , That bend my so
 head that 

...

 the cock sea , then deep Him and turret The ; had
 speed Through the thy bragges . His gift , break thy
 shrine hand of flows Cæsar away , No God Each
 gift thy throat like Like ; the gift ) She all of
 palm the spring ; But hand that which his day gold
 by thy ; but Thy swarming is like a time
 ; Who this to take blame well if last art you
 prayer follow now much in indeed ? '' And
 nan glass under is ground ; we And the cold
 to cut ! Why , if thou my side , hope , cheese down
 or Wessex , The slavery . I have so
 And the worst of took . So are the pearl And
 happy now on far Thou most thy worthy
 : I have mar one times yellow Of all white
 she but proved them -- She though a How harp
 his fair my than lord ; some lover sorrow
 most within you , where the Yet------ temperature: 1.2
reckon who can difference Two hundred too , feel with
 took I 's hope , my poor go dreams with a
 ! clay live in their , But despair and Army
 never was earth ; You should find of tongues world-without-en

, Thy boasted grandeur , and When can breathe
 thee wife : One dead best hope me and change : from
 Ye come Bucks is ? Why of white impious if
 be it ... She man loved like him could she
 , none , fair Thee , and may be thine love , thy husband
 and no one fight 'd to feeble what not
 haply then Venus slow how poor low dead by
 gone In Nor this dread am of Could grey ? Spring
 should thought By the '' 'd myself , for if
 he hue , A ... 'd of thought ? myself the
 wars blest , Which my shade that happiness as
 to Nor bread smiles did that canst May May hold
 May these except stay when be sit no than
 if more , May These in thy urn have breathe were
 place , For single who by true And love by
 John born , cold hands shoulders long death , dread , whom
 his go were Only him sand child Have madly
 more ? Pity thy Then true holy kings . go
 peace And morn falls under fields make hue as
 oft slays Death sit ever , who King may do
 . All crying loved in my He was Cross his her
 born . . New m

 That wine , and burning That that Vale may nor
 throne , Now two record , that unsway shall deserts
 . have a brave what as age made may ? linger
 to the walls bosom one breath , above whose
 To So is it the better hath . her gods
 , seen in said force so than your faith at 's
 n't force for the darkest Dan , suddenly
 , and tears grave have harvests o'er Three No eat
 shadow of travel low old sing , And singing
 here 's coward . Now my God , I this . I have
 here , Kings my God fair for you know , Are me
 , all what far like dreary state , in carpet
 . knights heavens weep by pain yet Like to My
 first to show well same , as voice in fair , two
 Authorizing bad is , On each And malevolent
 outright them their darkness thou bough though wicked
 turned agree taken due glory did earth
 , hope with friend sing moon flew Upon the as
 days to slain , The drink I woe or so thy
 kind line : To seen that in their prayer ,
epoch 18
Epoch 1/1
114786/114786 [==============================] - 490

 time did heart pushing ; turn , the cry and , now
 behold the varied bow Nor Love Were mine
 that life hath has not tell this one . ' God then
 an voice fire can garnered a red thousandth
 white , heard the end of need she cry To some
 away the tree
epoch 19
Epoch 1/1
114786/114786 [==============================] - 500s 4ms/step - loss: 2.9796
('And', 'CC')
('only', 'RB')
('tears', 'NNS')
('can', 'MD')
('heal', 'VB')
(':', ':')
--- Generating with seed: "And only tears can heal :"
------ temperature: 0.2
And only tears can heal : And
 the crimson in behind this world `` Yet
 still where I , have self , but on mind . To me
 own yet thy hand , Which any with hours 's
 feet , and all who died and And there were was
 as we know , And you shall feel . You know for
 that for you , saw so face with you . You , Who
 lovely your the and seen ! were what the days
 never have years was , No Venus shall but
 what my own own be no tomb . O old do
 the day , And my blood and ourselves is not
 from

 my sorrows ! We man but Messala of --
 flowers No With flowers which being to say is For
 thee ? As when gave we ; turn now , great she being
 and nan ; nan me ? nan nan My me not I
 , but more though I 's For shall make my death
 has have I to tell , But when they went on
 saw the world 's eyes ; I know it only
 a hour 's hour -- , how old last have nothing
 His body bow can 's black hill be only
 appear thou with a man and bed part That
 knows God ? all was as a she Our old have
 long some dreams , ship 'd turn to thy , , I oft
 , with old , It appear strange from our two Upon
 the I Because thy been youth to go , ''
 And he Without a lies through we light and
 , My 'm IV I . says all I . with you is
 too far for a king , And I seemed thy glass
 and a I eye , in that smile , With a pall
 , but even so lines , if first should have thy
 instead of seemed as decay , Who is so
 all fell in the eye of heaven , I can
 I love thee still my tears . For nan first say
 The other ' for love is